In [1]:
from dask_kubernetes.operator import KubeCluster
from dask_kubernetes.operator.kubecluster.kubecluster import CreateMode

In [54]:
cluster = KubeCluster.from_name("evaluator-lw-iris2")

Output()

In [55]:
cluster

KubeCluster(evaluator-lw-iris2, 'tcp://localhost:63122', workers=40, threads=40, memory=160.00 GiB)

In [56]:
cluster

KubeCluster(evaluator-lw-iris2, 'tcp://localhost:63122', workers=40, threads=40, memory=160.00 GiB)

In [6]:
cluster.scale(1, worker_group="default")

In [7]:
cluster

KubeCluster(evaluator-lw-iris2, 'tcp://localhost:63930', workers=66, threads=66, memory=264.00 GiB)

In [12]:
cluster.scale(40, worker_group="cpu")

In [13]:
cluster

KubeCluster(evaluator-lw-iris2, 'tcp://localhost:63930', workers=40, threads=40, memory=160.00 GiB)

In [14]:
dir(cluster)

['_Cluster__asynchronous',
 '_Cluster__loop',
 '__aenter__',
 '__aexit__',
 '__annotations__',
 '__await__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt',
 '_adaptive',
 '_add_worker_group',
 '_cached_widget',
 '_close',
 '_cluster_class_name',
 '_cluster_info',
 '_cluster_manager_logs',
 '_connect_cluster',
 '_create_cluster',
 '_custom_cluster_spec',
 '_delete_worker_group',
 '_get_logs',
 '_get_scheduler_address',
 '_instances',
 '_ipython_display_',
 '_log',
 '_loop_runner',
 '_repr_html_',
 '_resource_timeout',
 '_rich_spinner',
 '_scale',
 '_scaling_status',
 '_show_rich_output',
 '_start',
 '_st

In [15]:
client = cluster.get_client()

/Users/lily/micromamba/envs/evaluator-test-env-openff/lib/python3.11/site-packages/distributed/client.py:1392: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| toolz   | 1.0.0  | 0.12.0    | 0.12.0  |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))
2025-03-20 16:14:27,612 - distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client


In [16]:
client

<Client: 'tcp://10.244.245.209:8786' processes=40 threads=40, memory=160.00 GiB>

In [18]:
cluster.requested

AttributeError: 'KubeCluster' object has no attribute 'workers'

In [19]:
cluster.resources

Box({'limits': {'cpu': '1', 'ephemeral-storage': '20.000Gi', 'memory': '4.000Gi', 'nvidia.com/gpu': '1'}, 'requests': {'cpu': '1', 'ephemeral-storage': '20.000Gi', 'memory': '4.000Gi', 'nvidia.com/gpu': '1'}})

In [57]:
import yaml

with open("default-worker.yaml", "r") as f:
    default_spec = yaml.load(f, yaml.FullLoader)

In [58]:
default_spec

{'replicas': 2,
 'spec': {'containers': [{'args': ['dask-worker',
     '--name',
     '$(DASK_WORKER_NAME)',
     '--dashboard',
     '--dashboard-address',
     '8788',
     '--resources',
     'GPU=1,notGPU=0'],
    'env': [{'name': 'OE_LICENSE', 'value': '/secrets/oe_license.txt'},
     {'name': 'DASK_DISTRIBUTED__WORKER__DAEMON', 'value': 'False'},
     {'name': 'DASK_LOGGING__DISTRIBUTED', 'value': 'debug'},
     {'name': 'DASK__TEMPORARY_DIRECTORY', 'value': '/evaluator-storage'},
     {'name': 'STORAGE_DIRECTORY', 'value': '/evaluator-storage'}],
    'image': 'ghcr.io/lilyminium/openff-images:tmp-evaluator-dask-v5',
    'name': 'worker',
    'ports': [{'containerPort': 8788,
      'name': 'http-dashboard',
      'protocol': 'TCP'}],
    'resources': {'limits': {'cpu': '1',
      'ephemeral-storage': '20.000Gi',
      'memory': '8.000Gi',
      'nvidia.com/gpu': '1'},
     'requests': {'cpu': '1',
      'ephemeral-storage': '20.000Gi',
      'memory': '8.000Gi',
      'nvidia.com

In [59]:
cluster.add_worker_group("default", custom_spec=default_spec)

In [24]:
from dask_kubernetes.operator import make_worker_spec

In [25]:
make_worker_spec()

{'replicas': 3,
 'spec': {'containers': [{'name': 'worker',
    'image': 'ghcr.io/dask/dask:latest',
    'args': ['dask-worker',
     '--name',
     '$(DASK_WORKER_NAME)',
     '--dashboard',
     '--dashboard-address',
     '8788'],
    'env': None,
    'resources': None,
    'ports': [{'name': 'http-dashboard',
      'containerPort': 8788,
      'protocol': 'TCP'}]}]}}

In [50]:
cluster.scale(1)